In [ ]:
import random
import sqlite3

def create_card_with_alghoritm_luna():
    card_example = '400000'
    card_random = ''.join(random.sample(['0','1','2','3','4','5','6','7','8','9'], 9))
    card = card_example + card_random
    s = 0
    for i in range(15):
        dig = int(card[i])
        if i%2 == 0:
            dig *= 2
            if dig > 9:
                dig -= 9
        s += dig
    last_digit = (10 - s%10)%10
    card += str(last_digit)
    pin = ''.join(random.sample(['0','1','2','3','4','5','6','7','8','9'], 4))
    return card, pin

def check_card_for_algorithm_luna(card):
    s = 0
    for i in range(15):
        dig = int(card[i])
        if i%2 == 0:
            dig *= 2
            if dig > 9:
                dig -= 9
        s += dig
    s += int(card[len(card) - 1])
    if s%10 == 0:
        return True
    else:
        return False

if __name__ == '__main__':
    #create/connection with database
    conn = sqlite3.connect('card.s3db')
    cur = conn.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS card(
    id INTEGER,
    number TEXT,
    pin TEXT,
    balance INTEGER
    );""")
    conn.commit()
    #
    #
    flag = 0
    while True:
        if flag == 0:
            result = input("1. Create an account\n2. Log into account\n0. Exit\n")
            if result == '1':
                #create number and pin to card
                card, pin = create_card_with_alghoritm_luna()
                print('Your card has been created')
                print('Your card number:')
                print(card)
                print('Your card PIN:')
                print(pin)
                #create new id for the database
                cur.execute('SELECT * FROM card;')
                id = cur.fetchall()
                if id == []:
                    id = 1
                else:
                    id = id[len(id) - 1][0] + 1
                #add card and pin for database
                cur.execute('INSERT INTO card VALUES (?, ?, ?, ?);', (id, card, pin, 0))
                conn.commit()
            elif result == '2':
                card_input = input('Enter your card number:')
                pin_input = input('Enter your PIN:')
                #search card and pin input in database
                cur.execute("SELECT * FROM card;")
                cards_and_pins = cur.fetchall()
                flag_in_search = 0
                for card_and_pin in cards_and_pins:
                    if card_input in card_and_pin and pin_input in card_and_pin:
                        flag_in_search = 1
                        balance = card_and_pin[3]
                if flag_in_search == 1:
                    print('You have successfully logged in!')
                    flag = 1
                else:
                    print('Wrong card number or PIN!')
            else:
                print('Bye!')
                break
        else:
            result = input('1. Balance\n2. Add income\n3. Do transfer\n4. Close account\n5. Log out\n0. Exit\n')
            if result == '1':
                print('Balance: ', balance)
            elif result == '2':
                add_balance = int(input('Enter income:'))
                balance += add_balance
                cur.execute(f'UPDATE card SET balance = {balance} WHERE number = {card_input};')
                conn.commit()
                print('Income was added!')
            elif result == '3':
                cur.execute("SELECT * FROM card;")
                cards_and_pins = cur.fetchall()
                transfer_card = input('Transfer\nEnter card number:\n')
                if not check_card_for_algorithm_luna(transfer_card):
                    print('Probably you made a mistake in the card number. Please try again!')
                else:
                    if transfer_card == card_input:
                        print("You can't transfer money to the same account!")
                    else:
                        flag_transfer = 0
                        for card_and_pin in cards_and_pins:
                            if transfer_card in card_and_pin:
                                balance_transfer_card = card_and_pin[3]
                                flag_transfer = 1
                        if flag_transfer == 0:
                            print('Such a card does not exist.')
                        else:
                            add_sum = int(input('Enter how much money you want to transfer:'))
                            if add_sum > balance:
                                print('Not enough money!')
                            else:
                                cur.execute(f'UPDATE card SET balance = {balance - add_sum} WHERE number = {card_input};')
                                conn.commit()
                                cur.execute(f'UPDATE card SET balance = {balance_transfer_card + add_sum} WHERE number = {transfer_card};')
                                conn.commit()
                                balance -= add_sum
                                print('Success!')
            elif result == '4':
                cur.execute(f'DELETE FROM card WHERE number = {card_input}')
                conn.commit()
                cur.execute("SELECT * FROM card;")
                cards_and_pins = cur.fetchall()
                conn.commit()
                print('The account has been closed!')
                flag = 0
            elif result == '5':
                flag = 0
            else:
                print('Bye!')
                break